file path for csv of interest: `'Data/crypto_data.csv'` , `'Data/sp500_data'` , `'Data/coin_list.csv'`

In [1]:
#conditional imports, ask user if they would like to update the data
import pandas
from Data.crypto_data import crypto_data_collector
import Data.sp500_data


    

In [4]:
user_start_date = '2013-01-01'
user_end_date = '2021-05-14'

#if user inputs 'y', API calls made to Alpha Vantage and Coin Gecko to collect and overwrite existing data, else 
reload_response = input('Would you like to reload (y or n)?')

if reload_response == 'y':
    crypto_price_df = crypto_data_collector(user_start_date, user_end_date) # only way to save a df from a function https://stackoverflow.com/questions/45579525/returning-a-dataframe-in-python-function
    print('crypto_price_data.csv saved, dataframe loaded with datetime index as `coin_historical_df`')
    crypto_price_df = coin_historical_df.copy()
else: #load the csv's in the Data folder
    

Would you like to reload (y or n)? y


coin_list.csv saved in Data/ folder, please wait ~10 mins while crypto_price_data is loading...
crypto_price_data.csv saved, dataframe loaded with datetime index as `coin_historical_df`


In [5]:
crypto_price_df.head()

,bitcoin prices,ethereum prices,binancecoin prices,cardano prices,dogecoin prices,ripple prices,tether prices,polkadot prices,bitcoin-cash prices,litecoin prices,uniswap prices,chainlink prices,usd-coin prices,stellar prices,solana prices,ethereum-classic prices,vechain prices,matic-network prices,eos prices,theta-token prices
time,,,,,,,,,,,,,,,,,,,,
2013-04-28,135.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.29983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,141.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.35940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-30,135.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.18295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-01,117.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.64914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-02,103.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.38879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3